In [1]:
%load_ext Cython

In [2]:
import numpy as np
from numpy import linalg as ln
from scipy import linalg
import random
import math
from math import ceil as ceil
from math import floor as floor
import matplotlib.pyplot as plt
import numba
import scipy.integrate
from scipy.integrate import quad
import timeit

In [4]:
@numba.jit
def energycal(J,h,x1,x2):
    return -(J*x1*x2)-h*x1
@numba.jit
def approxfun(x,x1,x2,y1,y2):
    return y1+(y2-y1)*(x-x1)/(x2-x1)

def process(times,trials,u,J,h,omega,rot):
    polymer=[]
    for i in range(1000):
        polymer.append(1-2*random.randint(0,1))
    t=0
    requirementcheck=999999
    timerr=0
    timebb=0
    timebr=0
    timerb=0
    tregister=[]
    corrlengtcoll=[]
    timeinvertvalsameup=[]
    timeinvertvalsamedown=[]
    timeinvertvaldiffup=[]
    timeinvertvaldiffdown=[]
    count=1000
    addmove=0
    remove=0
    blue=0
    red=0
    velocity=[]
    currentrrtorr=[]
    currentrrtorb=[]
    currentrbtobr=[]
    currentrbtobb=[]
    currentbbtobr=[]
    currentbbtobb=[]  
    currentbrtorr=[]
    currentbrtorb=[]
    propi=list(np.zeros((1000,2),dtype=np.float64))
    timeaddremovecollect=list(np.zeros((1000,4),dtype=np.float64))
    magnet=list(np.zeros((1000,2),dtype=np.float64))
    proprr=list(np.zeros((1000),dtype=np.float64))
    proprb=list(np.zeros((1000),dtype=np.float64))
    propbr=list(np.zeros((1000),dtype=np.float64))
    propbb=list(np.zeros((1000),dtype=np.float64))
    timefrombb=list(np.zeros((1000,4),dtype=np.float64))
    timefromrr=list(np.zeros((1000,4),dtype=np.float64))
    timefromrb=list(np.zeros((1000,4),dtype=np.float64))
    timefrombr=list(np.zeros((1000,4),dtype=np.float64))
    overmag=0
    for i in range (1000):
        timeinvertvalsameup.append([(i+1),quad(lambda x:math.exp(-J-h*math.cos(x*2*math.pi*rot/omega)),0,omega*(i+1)/1000)[0]])
        timeinvertvalsamedown.append([(i+1),quad(lambda x:math.exp(-J+h*math.cos(x*2*math.pi*rot/omega)),0,omega*(i+1)/1000)[0]])
        timeinvertvaldiffup.append([(i+1),quad(lambda x:math.exp(J-h*math.cos(x*2*math.pi*rot/omega)),0,omega*(i+1)/1000)[0]])
        timeinvertvaldiffdown.append([(i+1),quad(lambda x:math.exp(J+h*math.cos(x*2*math.pi*rot/omega)),0,omega*(i+1)/1000)[0]])    
    mueq=math.log(2/(math.exp(J)*(math.cosh(h)+math.sqrt(math.exp(-4*J)+math.sinh(h)**2))))
    mueq0=math.log(2/(math.exp(J)*(math.cosh(0)+math.sqrt(math.exp(-4*J)+math.sinh(0)**2))))
    for i in range(count):
        tregister.append(0)
        if polymer[i]>0:
            red=red+1
        else:
            blue=blue+1
    del tregister[-1]
    for k in range (trials):
        if count>50000000:
            break
        tnon=(t%((omega)))*1000/omega
        tadd=-math.log(random.random())/(math.exp(u+mueq0))
        if polymer[count-1]*polymer[count-2]>0:
            if polymer[count-1]>0:
                randomtime=-math.log(random.random())
                if tnon<timeinvertvalsameup[0][0]:
                    functimezero=approxfun(tnon,0,1,0,timeinvertvalsameup[0][1])
                else:
                    nexttozero=next(x[0] for x in timeinvertvalsameup if x[0] > tnon)
                    functimezero=approxfun(tnon,(nexttozero-1),nexttozero,timeinvertvalsameup[nexttozero-2][1],
                                       timeinvertvalsameup[nexttozero-1][1])
                randomtime=randomtime+functimezero
                counter=0
                trem=0
                while randomtime>timeinvertvalsameup[-1][1]:
                    counter=counter+1
                    randomtime=randomtime-timeinvertvalsameup[-1][1]
                    trem=trem+omega
                if randomtime<timeinvertvalsameup[0][1]:
                    functiontimenext=approxfun(randomtime,0,
                                           timeinvertvalsameup[0][1],0,1)    
                else:
                    nextotime=next(x[0] for x in timeinvertvalsameup if x[1] > randomtime)
                    functiontimenext=approxfun(randomtime,timeinvertvalsameup[nextotime-2][1],
                                           timeinvertvalsameup[nextotime-1][1],nextotime-1,nextotime)
                trem=trem+(functiontimenext-tnon)*(omega)/1000
            else:
                randomtime=-math.log(random.random())
                if tnon<timeinvertvalsamedown[0][0]:
                    functimezero=approxfun(tnon,0,1,0,timeinvertvalsamedown[0][1])
                else:
                    nexttozero=next(x[0] for x in timeinvertvalsamedown if x[0] > tnon)
                    functimezero=approxfun(tnon,(nexttozero-1),nexttozero,timeinvertvalsamedown[nexttozero-2][1],
                                       timeinvertvalsamedown[nexttozero-1][1])
                randomtime=randomtime+functimezero
                counter=0
                trem=0
                while randomtime>timeinvertvalsamedown[-1][1]:
                    counter=counter+1
                    randomtime=randomtime-timeinvertvalsamedown[-1][1]
                    trem=trem+(omega)
                if randomtime<timeinvertvalsamedown[0][1]:
                    functiontimenext=approxfun(randomtime,0,
                                           timeinvertvalsamedown[0][1],0,1)    
                else:
                    nextotime=next(x[0] for x in timeinvertvalsamedown if x[1] > randomtime)
                    functiontimenext=approxfun(randomtime,timeinvertvalsamedown[nextotime-2][1],
                                           timeinvertvalsamedown[nextotime-1][1],nextotime-1,nextotime)
                trem=trem+(functiontimenext-tnon)*(omega)/1000
        else:
            if polymer[count-1]>0:
                randomtime=-math.log(random.random())
                if tnon<timeinvertvaldiffup[0][0]:
                    functimezero=approxfun(tnon,0,1,0,timeinvertvaldiffup[0][1])
                else:
                    nexttozero=next(x[0] for x in timeinvertvaldiffup if x[0] > tnon)
                    functimezero=approxfun(tnon,(nexttozero-1),nexttozero,timeinvertvaldiffup[nexttozero-2][1],
                                       timeinvertvaldiffup[nexttozero-1][1])
                randomtime=randomtime+functimezero
                counter=0
                trem=0
                while randomtime>timeinvertvaldiffup[-1][1]:
                    counter=counter+1
                    randomtime=randomtime-timeinvertvaldiffup[-1][1]
                    trem=trem+(omega)
                if randomtime<timeinvertvaldiffup[0][1]:
                    functiontimenext=approxfun(randomtime,0,
                                           timeinvertvaldiffup[0][1],0,1)    
                else:
                    nextotime=next(x[0] for x in timeinvertvaldiffup if x[1] > randomtime)
                    functiontimenext=approxfun(randomtime,timeinvertvaldiffup[nextotime-2][1],
                                           timeinvertvaldiffup[nextotime-1][1],nextotime-1,nextotime)
                trem=trem+(functiontimenext-tnon)*(omega)/1000
            else:
                randomtime=-math.log(random.random())
                if tnon<timeinvertvaldiffdown[0][0]:
                    functimezero=approxfun(tnon,0,1,0,timeinvertvaldiffdown[0][1])
                else:
                    nexttozero=next(x[0] for x in timeinvertvaldiffdown if x[0] > tnon)
                    functimezero=approxfun(tnon,(nexttozero-1),nexttozero,timeinvertvaldiffdown[nexttozero-2][1],
                                       timeinvertvaldiffdown[nexttozero-1][1])
                randomtime=randomtime+functimezero
                counter=0
                trem=0
                while randomtime>timeinvertvaldiffdown[-1][1]:
                    counter=counter+1
                    randomtime=randomtime-timeinvertvaldiffdown[-1][1]
                    trem=trem+(omega)
                if randomtime<timeinvertvaldiffdown[0][1]:
                    functiontimenext=approxfun(randomtime,0,
                                           timeinvertvaldiffdown[0][1],0,1)    
                else:
                    nextotime=next(x[0] for x in timeinvertvaldiffdown if x[1] > randomtime)
                    functiontimenext=approxfun(randomtime,timeinvertvaldiffdown[nextotime-2][1],
                                           timeinvertvaldiffdown[nextotime-1][1],nextotime-1,nextotime)
                trem=trem+(functiontimenext-tnon)*(omega)/1000
                
        if tadd>trem:
            timeaddremovecollect[floor(tnon)][2]=timeaddremovecollect[floor(tnon)][2]+trem
            timeaddremovecollect[floor(tnon)][3]=timeaddremovecollect[floor(tnon)][3]+1
            if polymer[-1]>0:
                if polymer[-2]>0:
                    timerr=timerr+trem
                    tmarker=trem
                    tstart=0
                    while tmarker>0:
                        proprr[int((tnon+tstart)%omega*1000/omega)]=+min(omega/1000,tmarker)
                        tmarker=tmarker-omega/1000
                        tstart=tstart+omega/1000

                else:
                    timebr=timebr+trem
                    tmarker=trem
                    tstart=0
                    while tmarker>0:
                        propbr[int((tnon+tstart)%omega*1000/omega)]=+min(omega/1000,tmarker)
                        tmarker=tmarker-omega/1000
                        tstart=tstart+omega/1000
   
            else:
                if polymer[-2]>0:
                    timerb=timerb+trem
                    tmarker=trem
                    tstart=0
                    while tmarker>0:
                        proprb[int((tnon+tstart)%omega*1000/omega)]=+min(omega/1000,tmarker)
                        tmarker=tmarker-omega/1000
                        tstart=tstart+omega/1000

                else:
                    timebb=timebb+trem
                    tmarker=trem
                    tstart=0
                    while tmarker>0:
                        propbb[int((tnon+tstart)%omega*1000/omega)]=+min(omega/1000,tmarker)
                        tmarker=tmarker-omega/1000
                        tstart=tstart+omega/1000

            t=t+trem
            count=count-1
            remove=remove+1
            overmag=overmag-polymer[-1]
            magnet[floor(tnon)][0]=magnet[floor(tnon)][0]+overmag
            magnet[floor(tnon)][1]=magnet[floor(tnon)][1]+1
            if polymer[-1]>0:
                red=red-1
            else:
                blue=blue-1
            del polymer[-1]
            del tregister[-1]
        else:
            timeaddremovecollect[floor(tnon)][0]=timeaddremovecollect[floor(tnon)][0]+tadd
            timeaddremovecollect[floor(tnon)][1]=timeaddremovecollect[floor(tnon)][1]+1
            if polymer[-1]>0:
                if polymer[-2]>0:
                    timerr=timerr+tadd
                    tmarker=tadd
                    tstart=0
                    while tmarker>0:
                        proprr[int((tnon+tstart)%omega*1000/omega)]=+min(omega/1000,tmarker)
                        tmarker=tmarker-omega/1000
                        tstart=tstart+omega/1000                    

                else:
                    timebr=timebr+tadd
                    tmarker=tadd
                    tstart=0
                    while tmarker>0:
                        propbr[int((tnon+tstart)%omega*1000/omega)]=+min(omega/1000,tmarker)
                        tmarker=tmarker-omega/1000
                        tstart=tstart+omega/1000              

            else:
                if polymer[-2]>0:
                    timerb=timerb+tadd
                    tmarker=tadd
                    tstart=0
                    while tmarker>0:
                        proprb[int((tnon+tstart)%omega*1000/omega)]=+min(omega/1000,tmarker)
                        tmarker=tmarker-omega/1000
                        tstart=tstart+omega/1000                     

                else:
                    timebb=timebb+tadd
                    tmarker=tadd
                    tstart=0
                    while tmarker>0:
                        propbb[int((tnon+tstart)%omega*1000/omega)]=+min(omega/1000,tmarker)
                        tmarker=tmarker-omega/1000
                        tstart=tstart+omega/1000             

            t=t+tadd
            count=count+1
            addmove=addmove+1
            tregister.append((t%((omega)))*1000/omega)
            polymer.append(1-2*random.randint(0,1))
            magnet[floor(tnon)][0]=magnet[floor(tnon)][0]+overmag
            magnet[floor(tnon)][1]=magnet[floor(tnon)][1]+1
            if polymer[-1]>0:
                red=red+1
            else:
                blue=blue+1

        if k%1000==0:
            velocity.append([t,count])
        timetotal=timerr+timebb+timebr+timerb

    for j in range(len(tregister)-1000):
        if polymer[1000+j]>0:
            propi[floor(tregister[999+j])][0]=propi[floor(tregister[999+j])][0]+1
        else:
            propi[floor(tregister[999+j])][1]=propi[floor(tregister[999+j])][1]+1
        
    totalprop=[]
    currenttimeall=[]
    for j in range(1000):    
        totalprop.append(proprr[j]+proprb[j]+propbr[j]+propbb[j])
    proplist=[]
    currentlist=[]
    for j in range(1000):
        proplist.append([proprr[j]/totalprop[j],proprb[j]/totalprop[j],propbr[j]/totalprop[j],propbb[j]/totalprop[j]])
    timetotal=timerr+timebb+timebr+timerb       

    
    

    return (blue/red,addmove,remove,count,blue,red,timerr,timebb,timebr,timerb,timerr+timebb+timebr+timerb)

In [7]:
#times,trials,u,J,h,omega,rot
for i in range(1):
    print(process(1,400000,5,4,0.2,10,1))

(0.9992136156827501, 270684, 129316, 142368, 71156, 71212, 23737.265544543327, 23647.168243637057, 1178.5862360855688, 1189.2211708234152, 49752.241195089366)


In [8]:
timeit.timeit('process(1,400000,5,4,0.2,10,1)','from __main__ import process',number=1)

25.839809400000036

In [9]:
%timeit print(process(1,400000,5,4,0.2,10,1))

(1.065782416467348, 270754, 129246, 142508, 73523, 68985, 22953.291672831616, 24486.562381136166, 1207.4362401089616, 1137.524907857578, 49784.81520193432)
(0.9737602846342006, 270508, 129492, 142016, 70064, 71952, 23989.353539774733, 23315.9139868262, 1160.1180621348115, 1201.9404346203253, 49667.326023356065)
(0.9999718883969358, 270644, 129356, 142288, 71143, 71145, 23850.641838128937, 23694.07002196462, 1175.14690030855, 1191.788962923871, 49911.64772332598)
(1.0196120230228096, 270555, 129445, 142110, 71745, 70365, 23530.913060670202, 23881.89975942052, 1190.1559030132648, 1193.8893319152976, 49796.858055019286)
(1.034655893810417, 270011, 129989, 141022, 71712, 69310, 23142.175019460738, 24121.759369005023, 1206.9293894032412, 1166.9628823579292, 49637.82666022692)
(1.0267444341702823, 270735, 129265, 142470, 72175, 70295, 23476.200553221097, 23897.245562702486, 1195.5244723735445, 1171.979167172236, 49740.94975546936)
(0.997916490694597, 270460, 129540, 141920, 70886, 71034, 237